In [1]:
import os
import pandas as pd
import numpy as np
import settings
pd.set_option('display.max_colwidth', -1)
import json

In [2]:
df = pd.read_csv(os.path.join(settings.DATA_DIR, 'train.csv'))

In [3]:
df[['comment_text', 'target']].sample(10)

,comment_text,target
9151,The problem with social justice warriors is that they have been conditioned to see everything as some type for slight.,0.0
1001484,"Except he literally bowed to the dictator(king) of a Muslim country last week. The same one IIRC that he lambasted Obama for bowing to under similar circumstances. \n\nNo, we are not gaining respect, fear perhaps, but not respect.",0.0
1035563,"No, they are the smart ones who pay no attention to the media.",0.0
1631673,"that is called textbook projection. now go look the big word up. and CMD is my initials, Christopher Michael Dunn, as once again you go public to let us all know you're not only uneducated, but you're profoundly ignorant on any given issue you post on. It's why you run like an intellectually bankrupt coward from the simple question I asked you, you can't answer, as that would require intelligence.",0.5
1387831,"A couple of thoughts on this. 1. While it may seem that Gov. Ige has not been doing much to firm up a base, I will tell you that his actions in regard to letting AG Chin run the whole immigration/refugee ban fight and the Governor's firm decision to not allow LNG to be the bridge fuel for Hawaii, but to go all renewable and double down on becoming 100% renewable is very telling as to whom he is trying to court for cash. That is progressive liberal organizations that not only support but promote such political philosophies. \n2. A more telling sign as to whom could win the Governors race next year is whichever candidate comes up with a cohesive, realistic vision of what Hawaii should be going into the future. Jack Burns did this in 1970 in defeating Gill in the Democratic Primary. Whomever comes up with the ""To Catch A Wave"" type promotion is the one that will be in the catbird seat moving forward.",0.0
70164,"The new-found levels of overnight solidarity between the base, Tea Party, and wave Republicans around Trump has been staggering, which includes Sanders voters. This, coupled with the Romney/#nevertrump momentum fizzling out is perhaps much of why Trump is now tied/ahead of Billary in the polls. I wish Trump well in defeating the pantsuit-wearing sociopath.",0.0
23403,"...or how about New York, jimb?",0.0
717606,"And what every you do, 907, do not investigate the negative dollar value from the pollution of naturally leaking coal seam bed methane; gas that is naturally leaking into Alaskan waters and into the atmosphere. OMG, it seems like it is almost everywhere in this State! By the way, please tell us about your environmental footprint. Sometimes talking about it instead of pointing fingers at others might make you less hypocritical and a more rounded person.",0.2
294267,"I gave you a thumbs-up because I mostly agree with you, except that I believe that in a way we do deserve what we get. Sadly, decisions like this are being made without taking our kids and grand kids into consideration. Going from bad to worse.",0.0
510878,Even if someone else is paying him...,0.0


In [5]:
df.rating.value_counts()

approved    1684758
rejected    120116 
Name: rating, dtype: int64

In [8]:
df['disagree_level'] = df.apply(lambda r: r['disagree'] > r['likes'], axis=1).astype(np.int)

In [9]:
df.disagree_level.value_counts()

0    1656124
1    148750 
Name: disagree_level, dtype: int64

In [10]:
from sklearn.preprocessing import LabelEncoder

In [11]:
df['rating'] = LabelEncoder().fit_transform(df['rating'])

In [12]:
df.rating.value_counts()

0    1684758
1    120116 
Name: rating, dtype: int64

In [4]:
AUX_COLUMNS = ['target', 'severe_toxicity', 'obscene', 'identity_attack', 'insult', 'threat']
df[AUX_COLUMNS].sample(10)

,target,severe_toxicity,obscene,identity_attack,insult,threat
1678328,0.000000,0.0,0.0,0.000000,0.000000,0.0
1714255,0.000000,0.0,0.0,0.000000,0.000000,0.0
116538,0.300000,0.0,0.0,0.000000,0.300000,0.0
1140418,0.000000,0.0,0.0,0.000000,0.000000,0.0
66948,0.900000,0.1,0.1,0.000000,0.800000,0.0
976023,0.000000,0.0,0.0,0.000000,0.000000,0.0
235638,0.000000,0.0,0.0,0.000000,0.000000,0.0
108996,0.000000,0.0,0.0,0.000000,0.000000,0.0
547120,0.166667,0.0,0.0,0.166667,0.166667,0.0
1147907,0.000000,0.0,0.0,0.000000,0.000000,0.0


In [9]:
print(len(df[df['target']>0.5]), len(df[df['severe_toxicity']>0.5]), len(df[df['obscene']>0.5]),len(df[df['identity_attack']>0.5]),len(df[df['insult']>0.5]), len(df[df['threat']>0.5]))

106438 8 7648 7633 79887 2793


In [10]:
len(df)

1804874

In [26]:
len(df[df['target']==0.5])

37896

In [15]:
len(df[(df['target']>0.5) & (df['insult']>0.5)])

78125

In [16]:
len(df[(df['target']>0.5) & (df['obscene']>0.5)])

7494

In [17]:
df['text_len'] = df.comment_text.map(lambda x: len(x.split(' ')))
len(df[df.text_len > 160])

88410

In [27]:
df_text = pd.read_csv(os.path.join(settings.DATA_DIR, 'test.csv'))
df_text.sample(10)

,id,comment_text
27652,7027652,tryanny doesn't mean what you think it means.
81418,7081418,"Get that you live to troll, Tommy.\n\nNot playing your silly games over the Christmas holidays."
14484,7014484,Except for the Christians in the southern hemisphere....
7792,7007792,NO'Leary !\n\nJust NO !!
83539,7083539,"Washington Post correction: (you can't make this stuff up and make it worse)\n\n\nEDITOR'S NOTE: This story has been updated to more precisely describe White House press secretary Sean Spicer's location late Tuesday night in the minutes before he briefed reporters. Spicer huddled with his staff among bushes near television sets on the White House grounds, not ""in the bushes,"" as the story originally stated.\n\n\nhttps://www.washingtonpost.com/news/post-politics/wp/2017/05/10/as-trump-fired-comey-his-staff-scrambled-to-explain-why/?utm_term=.12f7acda3700"
41306,7041306,"You my friend dream of the Utopia. We all want to live there, unfortunately it does not exist."
4440,7004440,Yes...they've gone crazy this year...good on ya!\n\nWell. Wait until NEXT year (*rolling my eyes*)
45371,7045371,"“Today, the Trump administration took up the cause of oppressed white people.”"
50534,7050534,"Mr. Clark has inadvertently stated the obvious...... ""Suddenly, he has to start thinking ""....\nWas J. Trudeau ""thinking"" when he ordered the removal of our F-18's ? Or was he ""playing to the crowd "". \nRussia has no intention of taking on ""the West"", Putin may be a cruel dictator, but he's not stupid. Russia was made aware of the air-strike at least 8 hours before. As opposed to Canada, one hour before. (yes, Putin was elected)\nOnce again Trudeau has been made irrelevant, and by extension, Canada."
31558,7031558,"Ms Smith: Our current health care system may be the most expensive in the industrialized world; but, at least it's also the least effective. Gary crum"


In [24]:
df_text['text_len'] = df_text.comment_text.map(lambda x: len(x.split(' ')))
len(df_text[df_text.text_len > 200])

62

In [38]:
import torch
from pytorch_pretrained_bert import BertTokenizer, BertModel, BertForMaskedLM

# OPTIONAL: if you want to have more information on what's happening, activate the logger as follows
import logging
logging.basicConfig(level=logging.INFO)

# Load pre-trained model tokenizer (vocabulary)
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Tokenized input
text = "[CLS] it's http://www.com Who was Jim Henson ? [SEP] Jim Henson was a puppeteer [SEP]"
tokenized_text = tokenizer.tokenize(text)
tokenized_text

INFO:pytorch_pretrained_bert.tokenization:loading vocabulary file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-vocab.txt from cache at /home/core/.pytorch_pretrained_bert/26bc1ad6c0ac742e9b52263248f6d0f00068293b33709fae12320c0e35ccfbbb.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084


['[CLS]',
 'it',
 "'",
 's',
 'http',
 ':',
 '/',
 '/',
 'www',
 '.',
 'com',
 'who',
 'was',
 'jim',
 'henson',
 '?',
 '[SEP]',
 'jim',
 'henson',
 'was',
 'a',
 'puppet',
 '##eer',
 '[SEP]']